# Финальное задание
# Студент 18ИАД - Ляшук Александр Юрьевич

## 1. Импортируем необходимые библиотеки и инициализируем PySpark

In [1]:
import sparkmonitor
import pandas as pd

from pyspark import SparkContext
from pyspark.sql import SparkSession

import warnings
warnings.simplefilter('ignore')

!jupyter nbextension install sparkmonitor --py --user --symlink 
!jupyter nbextension enable sparkmonitor --py --user            
!jupyter serverextension enable --py --user sparkmonitor
!ipython profile create && echo "c.InteractiveShellApp.extensions.append('sparkmonitor.kernelextension')" >>  $(ipython profile locate default)/ipython_kernel_config.py

conf.setAppName('final')
conf.setMaster('local[*]')
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession.builder.appName('final').getOrCreate()

all_scores = []

Installing /home/xperience/conda/lib/python3.6/site-packages/sparkmonitor/static -> sparkmonitor
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable sparkmonitor --user --py
    
Enabling notebook extension sparkmonitor/module...
      - Validating: OK
Enabling: sparkmonitor.serverextension
- Writing config: /home/xperience/.jupyter
    - Validating...
      sparkmonitor.serverextension  OK


## 2. Загрузка датасета в Pandas DataFrame

In [24]:
data = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', sep = ',')

In [25]:
data.head().T

,0,1,2,3,4
Unique Key,33490593,33490595,33490596,33490597,33490598
Created Date,06/01/2016 08:31:00 AM,06/01/2016 12:35:53 PM,06/01/2016 10:02:44 PM,06/01/2016 11:02:16 AM,06/01/2016 01:04:58 PM
Closed Date,06/02/2016 12:00:00 PM,07/13/2016 03:25:20 PM,06/06/2016 12:10:06 PM,06/14/2016 04:26:38 PM,06/01/2016 06:41:07 PM
Agency,DSNY,DOT,DOT,DFTA,NYPD
Agency Name,BCC - Queens East,Department of Transportation,Department of Transportation,Department for the Aging,New York City Police Department
Complaint Type,Missed Collection (All Materials),Sidewalk Condition,Sidewalk Condition,Alzheimer's Care,Blocked Driveway
Descriptor,2R Bulk-Missed Recy Collection,Sidewalk Violation,Sidewalk Violation,NaN,No Access
Location Type,Sidewalk,Sidewalk,Sidewalk,Senior Address,Street/Sidewalk
Incident Zip,11004,11212,11355,11249,10466
Incident Address,259-10 HILLSIDE AVENUE,1474 PITKIN AVENUE,57-15 157 STREET,NaN,2281 LIGHT STREET


## 3. Задача классификации - предсказать агенство по текстовому описанию проблемы

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Tokenizer, IDF, HashingTF

### 3.1 Извлечение из исходного датафрейма текстовое описание и название агенства

In [5]:
data = data[['Resolution Description', 'Agency']]

In [6]:
data.head()

,Resolution Description,Agency
0,The Department of Sanitation investigated this...,DSNY
1,A sidewalk dismissal inspection (work done by ...,DOT
2,A sidewalk dismissal inspection (work done by ...,DOT
3,The Department for the Aging contacted you and...,DFTA
4,The Police Department responded and upon arriv...,NYPD


### 3.2 Очистка записей, не имеющих описания

In [7]:
data = data.dropna(subset=['Resolution Description'])

### 3.3 LabelEncoder - кодирование названия классов

In [8]:
encoder = LabelEncoder()
data['Agency'] = encoder.fit_transform(data['Agency'])
data.head()

,Resolution Description,Agency
0,The Department of Sanitation investigated this...,14
1,A sidewalk dismissal inspection (work done by ...,12
2,A sidewalk dismissal inspection (work done by ...,12
3,The Department for the Aging contacted you and...,5
4,The Police Department responded and upon arriv...,19


### 3.4 Загрузка pandas Dataframe в Spark Dataframe и разбиение на тренировочную и тестовую выборки

In [9]:
data = spark.createDataFrame(data)

train, test = data.randomSplit([0.8, 0.2], seed=7)
train.show(5)

+----------------------+------+
|Resolution Description|Agency|
+----------------------+------+
|  A copy of the req...|     9|
|  A copy of the req...|     9|
|  A copy of the req...|     9|
|  A sidewalk dismis...|    12|
|  A sidewalk dismis...|    12|
+----------------------+------+
only showing top 5 rows



### 3.5 Инициализация модели логистической регрессии в Spark

In [10]:
lr = LogisticRegression(labelCol='Agency')

### 3.6 Представление текста в виде TF-iDF

In [11]:
tokenizer = Tokenizer(inputCol="Resolution Description", outputCol="tokenized")
hashingTF = HashingTF(inputCol="tokenized", outputCol="rawFeatures", numFeatures=50)
idf = IDF(inputCol="rawFeatures", outputCol="features")

train = tokenizer.transform(train)
test = tokenizer.transform(test)

train = hashingTF.transform(train)
test = hashingTF.transform(test)

idfModel = idf.fit(train)
train = idfModel.transform(train)
test = idfModel.transform(test)

### 3.7 Обучение модели

In [12]:
model = lr.fit(train)

In [13]:
preds = model.transform(test).toPandas()[['Resolution Description','Agency', 'prediction']]

In [14]:
preds['Agency'] = encoder.inverse_transform(preds['Agency'])
preds['prediction'] = encoder.inverse_transform([int(i) for i in preds['prediction']])

all_scores.append({
    "Type": "Agency", 
    "Accuracy": float(accuracy_score(preds['Agency'], preds['prediction']))
})

In [15]:
preds.drop_duplicates(subset=['Resolution Description']).head(20)

,Resolution Description,Agency,prediction
0,A copy of the requested document was mailed.,DOF,DOF
1,A sidewalk dismissal inspection (work done by ...,DOT,DOT
12,"Based on the information you provided, the Dep...",DHS,DHS
16,Callers should contact the DOHMH Foodborne Ill...,DOHMH,DOHMH
19,DOB investigated the complaint and Stop Work O...,DOB,DOB
20,General maintenance and cleaning is on a regul...,DOT,DOT
27,No action was taken because the Department of ...,DPR,DPR
30,No action was taken because the Department of ...,DPR,DPR
33,No action was taken because the agency determi...,DPR,DPR
34,Please submit the following document(s) so tha...,DOF,DOF


In [16]:
print('Доля правильных ответов на тестовой выборке составляет {:.2f}%'.format(float(accuracy_score(preds['Agency'], preds['prediction'])) * 100))

Доля правильных ответов на тестовой выборке составляет 99.90%


### 4. Задача классификации - предсказать тип жалобы по текстовому описанию проблемы

In [17]:
label = 'Complaint Type'

data = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', sep = ',')
data = data[['Resolution Description', label]]
data = data.dropna()
encoder = LabelEncoder()
data[label] = encoder.fit_transform(data[label])
data.head()

data = spark.createDataFrame(data)
train, test = data.randomSplit([0.8, 0.2], seed=7)

lr = LogisticRegression(labelCol=label)

tokenizer = Tokenizer(inputCol="Resolution Description", outputCol="tokenized")
hashingTF = HashingTF(inputCol="tokenized", outputCol="rawFeatures", numFeatures=50)
idf = IDF(inputCol="rawFeatures", outputCol="features")

train = tokenizer.transform(train)
test = tokenizer.transform(test)

train = hashingTF.transform(train)
test = hashingTF.transform(test)

idfModel = idf.fit(train)
train = idfModel.transform(train)
test = idfModel.transform(test)

model = lr.fit(train)
preds = model.transform(test).toPandas()[['Resolution Description',label, 'prediction']]
preds[label] = encoder.inverse_transform(preds[label])
preds['prediction'] = encoder.inverse_transform([int(i) for i in preds['prediction']])

all_scores.append({
    "Type": label, 
    "Accuracy": float(accuracy_score(preds[label], preds['prediction']))
})

print('Доля правильных ответов на тестовой выборке составляет {:.2f}%'.format(float(accuracy_score(preds[label], preds['prediction'])) * 100))
preds.drop_duplicates(subset=['Resolution Description']).head(20)

Доля правильных ответов на тестовой выборке составляет 56.38%


,Resolution Description,Complaint Type,prediction
0,A copy of the requested document was mailed.,DOF Parking - Request Copy,DOF Parking - Request Copy
1,A sidewalk dismissal inspection (work done by ...,Sidewalk Condition,Sidewalk Condition
12,"Based on the information you provided, the Dep...",Homeless Person Assistance,Homeless Person Assistance
16,Callers should contact the DOHMH Foodborne Ill...,Food Poisoning,Food Poisoning
19,DOB investigated the complaint and Stop Work O...,General Construction/Plumbing,General Construction/Plumbing
20,General maintenance and cleaning is on a regul...,Broken Muni Meter,Broken Muni Meter
27,No action was taken because the Department of ...,Illegal Tree Damage,Overgrown Tree/Branches
30,No action was taken because the Department of ...,Animal in a Park,Maintenance or Facility
33,No action was taken because the agency determi...,Maintenance or Facility,Maintenance or Facility
34,Please submit the following document(s) so tha...,SCRIE,SCRIE


### 5. Задача классификации - предсказать категорию жалобы по текстовому описанию проблемы

In [18]:
label = 'Descriptor'

data = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', sep = ',')
data = data[['Resolution Description', label]]
data = data.dropna()
encoder = LabelEncoder()
data[label] = encoder.fit_transform(data[label])
data.head()

data = spark.createDataFrame(data)
train, test = data.randomSplit([0.8, 0.2], seed=7)

lr = LogisticRegression(labelCol=label)

tokenizer = Tokenizer(inputCol="Resolution Description", outputCol="tokenized")
hashingTF = HashingTF(inputCol="tokenized", outputCol="rawFeatures", numFeatures=50)
idf = IDF(inputCol="rawFeatures", outputCol="features")

train = tokenizer.transform(train)
test = tokenizer.transform(test)

train = hashingTF.transform(train)
test = hashingTF.transform(test)

idfModel = idf.fit(train)
train = idfModel.transform(train)
test = idfModel.transform(test)

model = lr.fit(train)
preds = model.transform(test).toPandas()[['Resolution Description',label, 'prediction']]
preds[label] = encoder.inverse_transform(preds[label])
preds['prediction'] = encoder.inverse_transform([int(i) for i in preds['prediction']])

all_scores.append({
    "Type": label, 
    "Accuracy": float(accuracy_score(preds[label], preds['prediction']))
})

print('Доля правильных ответов на тестовой выборке составляет {:.2f}%'.format(float(accuracy_score(preds[label], preds['prediction'])) * 100))
preds.drop_duplicates(subset=['Resolution Description']).head(20)

Доля правильных ответов на тестовой выборке составляет 41.21%


,Resolution Description,Descriptor,prediction
0,A copy of the requested document was mailed.,Decision and Order,10 Litter Basket / Request
1,A sidewalk dismissal inspection (work done by ...,Sidewalk Violation,Sidewalk Violation
12,Callers should contact the DOHMH Foodborne Ill...,3 or More,3 or More
13,DOB investigated the complaint and Stop Work O...,Working Contrary To Stop Work Order,Elevator - Defective/Not Working
14,DOB violation was issued for failing to comply...,Cons - Contrary/Beyond Approved Plans/Permits,Structure - Outdoors
15,General maintenance and cleaning is on a regul...,Coin or Card Did Not Register,Out of Order
23,No action was taken because the Department of ...,Bicycle Chained to Tree,Hitting Power/Phone Lines
26,No action was taken because the Department of ...,Grass/Weeds,Grass/Weeds
27,No action was taken because the agency determi...,Structure - Outdoors,Structure - Outdoors
29,Please submit the following document(s) so tha...,Application Renewal,Application Renewal


### 6. Задача классификации - предсказать категорию местоположения по текстовому описанию проблемы

In [19]:
label = 'Location Type'

data = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', sep = ',')
data = data[['Resolution Description', label]]
data = data.dropna()
encoder = LabelEncoder()
data[label] = encoder.fit_transform(data[label])
data.head()

data = spark.createDataFrame(data)
train, test = data.randomSplit([0.8, 0.2], seed=7)

lr = LogisticRegression(labelCol=label)

tokenizer = Tokenizer(inputCol="Resolution Description", outputCol="tokenized")
hashingTF = HashingTF(inputCol="tokenized", outputCol="rawFeatures", numFeatures=50)
idf = IDF(inputCol="rawFeatures", outputCol="features")

train = tokenizer.transform(train)
test = tokenizer.transform(test)

train = hashingTF.transform(train)
test = hashingTF.transform(test)

idfModel = idf.fit(train)
train = idfModel.transform(train)
test = idfModel.transform(test)

model = lr.fit(train)
preds = model.transform(test).toPandas()[['Resolution Description',label, 'prediction']]
preds[label] = encoder.inverse_transform(preds[label])
preds['prediction'] = encoder.inverse_transform([int(i) for i in preds['prediction']])

all_scores.append({
    "Type": label, 
    "Accuracy": float(accuracy_score(preds[label], preds['prediction']))
})

print('Доля правильных ответов на тестовой выборке составляет {:.2f}%'.format(float(accuracy_score(preds[label], preds['prediction'])) * 100))
preds.drop_duplicates(subset=['Resolution Description']).head(20)

Доля правильных ответов на тестовой выборке составляет 73.44%


,Resolution Description,Location Type,prediction
0,A sidewalk dismissal inspection (work done by ...,Sidewalk,Sidewalk
11,"Based on the information you provided, the Dep...",Other,Street/Sidewalk
16,Callers should contact the DOHMH Foodborne Ill...,Restaurant/Bar/Deli/Bakery,Restaurant/Bar/Deli/Bakery
18,General maintenance and cleaning is on a regul...,Highway,Street
27,No action was taken because the Department of ...,Park,Park
30,No action was taken because the agency determi...,Park,Park
33,Please submit the following document(s) so tha...,Senior Address,Senior Address
35,See notes.,"1-, 2- and 3- Family Home",Property Address
63,Thank you for your report. Your report will b...,Roadway,Roadway
65,Thank you for your report. The Department of H...,Commercial Property,Residential Property


### 7. Задача классификации - предсказать канал связи по текстовому описанию проблемы

In [20]:
label = 'Open Data Channel Type'

data = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', sep = ',')
data = data[['Resolution Description', label]]
data = data.dropna()
encoder = LabelEncoder()
data[label] = encoder.fit_transform(data[label])
data.head()

data = spark.createDataFrame(data)
train, test = data.randomSplit([0.8, 0.2], seed=7)

lr = LogisticRegression(labelCol=label)

tokenizer = Tokenizer(inputCol="Resolution Description", outputCol="tokenized")
hashingTF = HashingTF(inputCol="tokenized", outputCol="rawFeatures", numFeatures=50)
idf = IDF(inputCol="rawFeatures", outputCol="features")

train = tokenizer.transform(train)
test = tokenizer.transform(test)

train = hashingTF.transform(train)
test = hashingTF.transform(test)

idfModel = idf.fit(train)
train = idfModel.transform(train)
test = idfModel.transform(test)

model = lr.fit(train)
preds = model.transform(test).toPandas()[['Resolution Description',label, 'prediction']]
preds[label] = encoder.inverse_transform(preds[label])
preds['prediction'] = encoder.inverse_transform([int(i) for i in preds['prediction']])

all_scores.append({
    "Type": label, 
    "Accuracy": float(accuracy_score(preds[label], preds['prediction']))
})

print('Доля правильных ответов на тестовой выборке составляет {:.2f}%'.format(float(accuracy_score(preds[label], preds['prediction'])) * 100))
preds.drop_duplicates(subset=['Resolution Description']).head(20)

Доля правильных ответов на тестовой выборке составляет 68.81%


,Resolution Description,Open Data Channel Type,prediction
0,A copy of the requested document was mailed.,PHONE,PHONE
1,A sidewalk dismissal inspection (work done by ...,PHONE,PHONE
12,"Based on the information you provided, the Dep...",MOBILE,MOBILE
16,Callers should contact the DOHMH Foodborne Ill...,ONLINE,ONLINE
19,DOB investigated the complaint and Stop Work O...,UNKNOWN,ONLINE
20,General maintenance and cleaning is on a regul...,MOBILE,PHONE
27,No action was taken because the Department of ...,OTHER,PHONE
30,No action was taken because the Department of ...,ONLINE,PHONE
33,No action was taken because the agency determi...,PHONE,PHONE
34,Please submit the following document(s) so tha...,PHONE,PHONE


## 8. Итоговые результаты

In [21]:
all_scores = pd.DataFrame(all_scores)
all_scores = all_scores[['Type', 'Accuracy']]
all_scores = all_scores.sort_values('Accuracy', ascending=False)
all_scores

,Type,Accuracy
0,Agency,0.999026
3,Location Type,0.734373
4,Open Data Channel Type,0.688125
1,Complaint Type,0.563772
2,Descriptor,0.412064


In [22]:
print('\t Результаты:')
for i in range(len(all_scores)):
    print('{:.2f}% - доля правильных ответов на тестовой выборке в задаче классификации по типу {}.'.format(all_scores.iloc[i,1] * 100, all_scores.iloc[i,0], ))

	 Результаты:
99.90% - доля правильных ответов на тестовой выборке в задаче классификации по типу Agency.
73.44% - доля правильных ответов на тестовой выборке в задаче классификации по типу Location Type.
68.81% - доля правильных ответов на тестовой выборке в задаче классификации по типу Open Data Channel Type.
56.38% - доля правильных ответов на тестовой выборке в задаче классификации по типу Complaint Type.
41.21% - доля правильных ответов на тестовой выборке в задаче классификации по типу Descriptor.
